In [1]:
library(MCMCglmm)
library(lme4) #random effect
library(rptR) 

Warning message:
"package 'MCMCglmm' was built under R version 4.0.5"
Loading required package: Matrix

Loading required package: coda

Warning message:
"package 'coda' was built under R version 4.0.5"
Loading required package: ape

Warning message:
"package 'rptR' was built under R version 4.0.5"


In [2]:
install.packages("MCMCglmm")

Warning message:
"package 'MCMCglmm' is in use and will not be installed"


TODO
read  Hadfield (2010) for different values tahn default ones

# Download data

In [3]:
path_ = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_'
path_adapt = file.path(path_,'Adaptability')
path_model = file.path(path_adapt,'repeatability_model')
df = read.csv(file.path(path_adapt,'df_MVT_4repeatibility.csv'), header = TRUE, sep = ";")
df$HenID = as.factor(df$HenID)   
df$PenID = as.factor(df$PenID) 
df$TrackingSystemID = as.factor(df$TrackingSystemID) 
df$Treatment = as.factor(df$Treatment) 
#li_var_cont = c('ID0','ID1','ID2','ID3','ID4','ID5','ID6','ID7')
#df$maxsevGap0_divided_maxsev_round = round(df$maxsevGap0_divided_maxsev, 1)
#df$maxsevGap0_divided_maxsev_round_log <- log(df$maxsevGap0_divided_maxsev_round)
#print(as.data.frame(table(df$maxsevGap0_divided_maxsev_round)))
#df_ = df[,c(li_var_cont, 'maxsevGap0_divided_maxsev_round','maxsevGap0_divided_maxsev_round_log')] 
print(dim(df))
summary(df)

[1] 6232   62


     HenID          PenID         CLASS              TagID          
 hen_1  :  49   pen10  : 882   Length:6232        Length:6232       
 hen_10 :  49   pen11  : 882   Class :character   Class :character  
 hen_105:  49   pen8   : 784   Mode  :character   Mode  :character  
 hen_108:  49   pen9   : 783                                        
 hen_110:  49   pen4   : 764                                        
 hen_111:  49   pen12  : 731                                        
 (Other):5938   (Other):1406                                        
 FocalLegringName       R.Pen       InitialStartDate   Treatment  
 Length:6232        Min.   :1.000   Length:6232        OFH :3071  
 Class :character   1st Qu.:2.000   Class :character   TRAN:3161  
 Mode  :character   Median :2.000   Mode  :character              
                    Mean   :2.475                                 
                    3rd Qu.:3.000                                 
                    Max.   :4.000             

In [ ]:
#Please refer to Hadfield (2010) and the
#primary literature on priors. Note that the rcov 
#and random specification now both use “idh()” to specify
#the matrix format. This is equivalent to SAS’s banded
#main diagonal matrix and forces the 
#covariances to be zero.

yz.bivarE.mcmc<-MCMCglmm(cbind(y_std,z_std)~(trait-1),
random=~idh(trait):indiv,rcov=~idh(trait):units,
family=c("gaussian","gaussian"), prior=prior.bivar,
nitt=130000,thin=100,burnin=30000, data=set3.data,verbose=FALSE)


In [22]:
#disadvantage of MCMCglmm: also requires consideration of how the prior distribution of variance components is modelled!!
y.mcmc = MCMCglmm(vertical_travel_distance_perh ~ DIB, random = ~HenID, family="gaussian", 
                  data = df_, verbose=FALSE)
summary(y.mcmc) 


 Iterations = 3001:12991
 Thinning interval  = 10
 Sample size  = 1000 

 DIC: 462.1526 

 G-structure:  ~HenID

      post.mean l-95% CI u-95% CI eff.samp
HenID     1.619    1.014    2.347     1000

 R-structure:  ~units

      post.mean l-95% CI u-95% CI eff.samp
units    0.3031   0.2496   0.3608    919.3

 Location effects: vertical_travel_distance_perh ~ 1 

            post.mean l-95% CI u-95% CI eff.samp  pMCMC    
(Intercept)     3.516    3.138    3.854     1000 <0.001 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

In [23]:
#repeatability can be calculated for each MCMC sample and the posterior mode and its 95% credibility interval then extracted:
rep.y = (y.mcmc$VCV[,"HenID"]/(y.mcmc$VCV[,"HenID"]+y.mcmc$VCV[,"units"]))
posterior.mode(rep.y) #repeatability estimate
HPDinterval(rep.y) #95% credibility interval

var1 
0.8419605

,lower,upper
var1,0.7717703,0.8987657


In [42]:
#from MCMCglmm (posterior modes between credibility intervals):
#β_0: -0.945 < -0.583 < -0.262 
posterior.mode(y.mcmc$Sol)
HPDinterval(y.mcmc$Sol)

(Intercept)         DIB 
-0.94049060  0.09942795

,lower,upper
(Intercept),-4.02823860,0.7745152
DIB,0.05343731,0.1452777


In [43]:
posterior.mode(y.mcmc$VCV) #HeniD: is the Vind (between individual variance), units is the Ve (within-individual variance)
HPDinterval(y.mcmc$VCV)

HenID     units 
1.5518865 0.2739214

,lower,upper
HenID,1.0093041,2.4206249
units,0.2353503,0.3467084


In [45]:
#OR via REML fitted model using the lme4 library
y.lme = lmer(vertical_travel_distance_perh~DIB+(1|HenID), data=df_)
summary(y.lme) 
#--> variance of HenID / (var(HENID)+Var(Residuel)) = 0,84

Linear mixed model fit by REML ['lmerMod']
Formula: vertical_travel_distance_perh ~ DIB + (1 | HenID)
   Data: df_

REML criterion at convergence: 567.2

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.26597 -0.54736  0.04465  0.54379  2.70051 

Random effects:
 Groups   Name        Variance Std.Dev.
 HenID    (Intercept) 1.5419   1.2417  
 Residual             0.2796   0.5288  
Number of obs: 251, groups:  HenID, 51

Fixed effects:
            Estimate Std. Error t value
(Intercept) -1.50052    1.24086  -1.209
DIB          0.09649    0.02363   4.083

Correlation of Fixed Effects:
    (Intr)
DIB -0.990

In [7]:
##raw repeatability: ignore temporal change
rep1 = rpt(vertical_travel_distance_perh ~ (1 | HenID), grname = "HenID", data = df,  datatype = "Gaussian", nboot = 10, npermut = 0) ###common week slope
print(rep1)

##conditional repeatability 1: shared change over time
rep2 = rpt(vertical_travel_distance_perh ~ WIB+ (1 | HenID), grname = "HenID", data = df, datatype = "Gaussian", nboot = 10, npermut = 0) ###individual differences in week slope
print(rep2)

##conditional repeatability 2: change over time that differs among individuals (this is what I want to get from the equivalent MCMCglmm model)
rep3 = rpt(vertical_travel_distance_perh ~ WIB+ (WIB | HenID), grname = "HenID", data = df,  datatype = "Gaussian", nboot = 10, npermut = 0)
print(rep3)

Bootstrap Progress:


Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.357
SE = 0.024
CI = [0.328, 0.395]
P  = 0 [LRT]
     NA [Permutation]

Bootstrap Progress:


Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.385
SE = 0.033
CI = [0.344, 0.445]
P  = 0 [LRT]
     NA [Permutation]



Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00256275 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00256275 (tol = 0.002, component 1)"


Bootstrap Progress:


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00835076 (tol = 0.002, component 1)"




Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.468
SE = 0.025
CI = [0.432, 0.499]
P  = 0 [LRT]
     NA [Permutation]



In [8]:
##raw repeatability: ignore temporal change
rep1 = rpt(vertical_travel_distance_perh ~ (1 | HenID), grname = "HenID", data = df,  datatype = "Gaussian", nboot = 10, npermut = 0) ###common week slope
print(rep1)

##conditional repeatability 1: shared change over time
rep2 = rpt(vertical_travel_distance_perh ~ DIB+ (1 | HenID), grname = "HenID", data = df, datatype = "Gaussian", nboot = 10, npermut = 0) ###individual differences in week slope
print(rep2)

##conditional repeatability 2: change over time that differs among individuals (this is what I want to get from the equivalent MCMCglmm model)
rep3 = rpt(vertical_travel_distance_perh ~ DIB+ (DIB | HenID), grname = "HenID", data = df,  datatype = "Gaussian", nboot = 10, npermut = 0)
print(rep3)

Bootstrap Progress:


Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.357
SE = 0.033
CI = [0.305, 0.402]
P  = 0 [LRT]
     NA [Permutation]

Bootstrap Progress:


Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.386
SE = 0.031
CI = [0.342, 0.432]
P  = 0 [LRT]
     NA [Permutation]



Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.0246584 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.0246584 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?"


Bootstrap Progress:


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.448913 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?"




Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.471
SE = 0.016
CI = [0.46, 0.507]
P  = 0 [LRT]
     NA [Permutation]



In [6]:
##raw repeatability: ignore temporal change
rep1 = rpt(vertical_travel_distance_perh ~ (1 | HenID), grname = "HenID", data = df_,  datatype = "Gaussian", nboot = 10, npermut = 0) ###common week slope
print(rep1)

##conditional repeatability 1: shared change over time
rep2 = rpt(vertical_travel_distance_perh ~ DIB+ (1 | HenID), grname = "HenID", data = df_, datatype = "Gaussian", nboot = 10, npermut = 0) ###individual differences in week slope
print(rep2)

##conditional repeatability 2: change over time that differs among individuals (this is what I want to get from the equivalent MCMCglmm model)
rep3 = rpt(vertical_travel_distance_perh ~ DIB+ (DIB | HenID), grname = "HenID", data = df_,  datatype = "Gaussian", nboot = 10, npermut = 0)
print(rep3)

Bootstrap Progress:
Bootstrap Progress:


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"


Bootstrap Progress:


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"




Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.838
SE = 0.036
CI = [0.785, 0.89]
P  = 2.65e-61 [LRT]
     NA [Permutation]



Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.846
SE = 0.032
CI = [0.786, 0.885]
P  = 4.46e-64 [LRT]
     NA [Permutation]



Repeatability estimation using the lmm method 

Repeatability for HenID
R  = 0.854
SE = 0.033
CI = [0.818, 0.909]
P  = 1.11e-62 [LRT]
     NA [Permutation]



In [14]:
###common week slope
MCMC_cond = MCMCglmm(vertical_travel_distance_perh ~WIB,
             random=~HenID,family=("gaussian"), #prior=prior1.1,
             nitt=110000,thin=25,burnin=10000, data=df, verbose=TRUE)


                       MCMC iteration = 0

                       MCMC iteration = 1000

                       MCMC iteration = 2000

                       MCMC iteration = 3000

                       MCMC iteration = 4000

                       MCMC iteration = 5000

                       MCMC iteration = 6000

                       MCMC iteration = 7000

                       MCMC iteration = 8000

                       MCMC iteration = 9000

                       MCMC iteration = 10000

                       MCMC iteration = 11000

                       MCMC iteration = 12000

                       MCMC iteration = 13000

                       MCMC iteration = 14000

                       MCMC iteration = 15000

                       MCMC iteration = 16000

                       MCMC iteration = 17000

                       MCMC iteration = 18000

                       MCMC iteration = 19000

                       MCMC iteration = 20000

                       MC

In [17]:
rep = (MCMC_cond$VCV[,"HenID"]/ (MCMC_cond$VCV[,"HenID"]+MCMC_cond$VCV[,"units"]))
posterior.mode(rep)
HPDinterval(rep)

var1 
0.3760644

,lower,upper
var1,0.3294679,0.4470789
